![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banner_Top_06.06.18.jpg?raw=true)


# Sustainability - A side-effect of globalization?

Globalization promises prosperity, however, it does not come without any effect on health of the natural environment. Economic activities across the globe have reduced earth's ability to provide necessary resources. [Global warming](https://en.wikipedia.org/wiki/Global_warming) is one of the major environmental concerns which has been largely influenced by the emission of greenhouse gases like carbon dioxide (CO<sub>2</sub>).

Let us try to find the connection between globalization and CO<sub>2</sub> emissions across the world. The [dataset](https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions) was obtained from [Our World in Data](https://ourworldindata.org/) which contains per capita CO<sub>2</sub> emission by various countries since early 1800s.


## Contribution of an average person in CO<sub>2</sub> emissions around the world

Let's check out how people in different countries contributed to the CO<sub>2</sub> emissions. Run the following cells to see the preview of the data and an animated choropleth map.

In [ ]:
# Import python libraries
import pandas as pd
import plotly_express as px
from ipywidgets import interact, fixed, widgets, Layout, Button, Box, fixed, HBox, VBox
import plotly.graph_objects as go

# Don't show warnings in output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import the dataset
df = pd.read_csv('../Data/co-emissions-per-capita.csv')
        
# Data clean up - keep rows pertaining to 1851-current and rename the columns
df = df[df['Year'] > 1850]\
     .rename(columns={'Code':'Country Code',
                         'Per capita CO₂ emissions (tonnes per capita)':'CO₂ emissions<br>(tonnes per capita)'})

# Display top 5 rows
df.head()

In [ ]:
# Plot animated choropleth map (Execution of this code cell will take few seconds)

fig = px.choropleth(df,   # dataframe with required data 
                    locations="Country Code",   # Column containing country codes
                    color="CO₂ emissions<br>(tonnes per capita)",   # Color of country should be based on per capita CO₂ emission
                    hover_name="Entity",   # Title to add to hover information
                    hover_data=["CO₂ emissions<br>(tonnes per capita)"],   # Data to add to hover information
                    color_continuous_scale=px.colors.sequential.Reds,   # Set the colorscale type
                    animation_frame = "Year",   # Values based on which animation frame will be prepared
                    range_color = [0,20],   # Range of the colorbar
                    
                    # Title of the chart
                    title = 'Per capita CO₂ emissions for countries around the world<br>\
Source: <a href="https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions">Our World in Data</a>'
                   )

# Show the figure
fig.show()

Click on *play* button and move your mouse around the map to see the per capita CO<sub>2</sub> emission by different countries.

### Questions
1. Before 1900, which country had highest per capita emission? Can you specify a reason?
2. As per the latest data, how would you rate Canada's per capita emission as compared to other populous countries?
3. Is inequality in per capita emission significant around the world? Is it one of the disadvantages of the globalization? 

## Share of countries around the world in global CO<sub>2</sub> emissions

Now that we know the per capita CO<sub>2</sub> emission for different countries, next question might be if this trend is same for the overall contribution of each country towards the global emission of carbon dioxide. Also, it would be beneficial to compare the share of different countires for different years.

Let's pull out the annual emission data from the same [source](https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions). Run the code blocks below to see the pie charts showing comparison for the years you choose. 

In [ ]:
# Import the dataset and remove rows with missing values
df2 = pd.read_csv('../Data/annual-co2-emissions-per-country.csv').dropna()

# Data clean up - keep rows pertaining to 1851-current and remove entities named 'World' and 'Micronesia'
df2 = df2[df2['Year'] > 1850][df2['Entity'] != 'World'][df2['Code'] != 'FSM']

# Display top 5 rows
df2.head()

In [ ]:
from plotly.subplots import make_subplots

# Define a callback function for "Show CO₂ Emissions" button
def compare_annual_emissions(ev):
    
    # Filter the rows for user-selected years
    subset1 = df2[df2['Year'] == year1_menu.value]
    subset2 = df2[df2['Year'] == year2_menu.value]
    
    
    # Make subplots
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    # Prepare text information to be shown when a pointer is hovered over pie chart
    subset1['text'] = subset1['Entity'] + '<br>' + 'Annual CO<sub>2</sub> Emission' \
                      + '<br>' + subset1['Annual CO₂ emissions (tonnes)'].astype(str) + ' tonnes'
        
    subset2['text'] = subset2['Entity'] + '<br>' + 'Annual CO<sub>2</sub> Emission' \
                      + '<br>' + subset2['Annual CO₂ emissions (tonnes)'].astype(str) + ' tonnes'
    
    # Add trace for each year's pie chart
    fig.add_trace(go.Pie(labels=subset1['Entity'],   # Labels for each portion in pie chart
                         values=subset1['Annual CO₂ emissions (tonnes)'],   # CO₂ emission in tonnes
                         name='Year<br>{}'.format(year1_menu.value),   # Which year the data is for
                         text=subset1['text'],   # Specify which text information to be shown
                         textinfo="none"),1,1)   # Disable default text information
    
    fig.add_trace(go.Pie(labels=subset2['Entity'],   # Labels for each portion in pie chart
                         values=subset2['Annual CO₂ emissions (tonnes)'],   # CO₂ emission in tonnes 
                         name='Year<br>{}'.format(year2_menu.value),   # Which year the data is for
                         text=subset2['text'],   # Specify which text information to be shown 
                         textinfo="none"),1,2)   # Disable default text information 
    
    # Use 'hole' to create a donut-like pie chart
    fig.update_traces(hole=.3, hoverinfo="text + percent + name")
        
    # Update the title of the figure
    fig.update_layout(title_text='Annual share of countries around the world in global CO₂ Emissions<br>\
Source: <a href="https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions">Our World in Data</a>')
    
    # Show the figure
    fig.show()

In [ ]:
# Layout for widgets
box_layout = Layout(display='flex', flex_flow='row', align_items='center', width='100%', justify_content = 'center')
style = {'description_width': 'initial'}

# Create dropdown menu for Year to be used for comparison
year1_menu = widgets.Dropdown(options = df2['Year'].sort_values().unique(), description ='Year 1: ', style = style, disabled=False)
year2_menu = widgets.Dropdown(options = df2['Year'].sort_values().unique(), description ='Year 2: ', style = style, disabled=False)

# Create "Show CO₂ Emissions" button and define click events
show_button = widgets.Button(button_style= 'info', description="Show CO₂ Emissions")
show_button.on_click(compare_annual_emissions)

# Define display order for the buttons and menus
display(Box(children = [year1_menu, year2_menu], layout = box_layout))
display(VBox(children = [show_button], layout = Layout(display= 'flex', flex_flow= 'column', align_items= 'center', width='100%', justify_content = 'center')))

Hover the mouse around to see the absolute/comparative annual contribution of the countries towards global CO<sub>2</sub> emission for the selected years. 

### Questions:
1. Compare the CO<sub>2</sub> emission of countries post World War I (Year 1919) and II (Year 1945). How the share of winners/losers changed?
2. Has top 5 contributors to global CO<sub>2</sub> emission changed in the era of contemporary globalization (i.e. from 1945 to 2017)?
3. What steps governments can/should take to reduce their share of CO<sub>2</sub> emission and ensure sustainable globalization? 

![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banners_Bottom_06.06.18.jpg?raw=true)
